# **Data Wrangling**

# 1. Imports, Options and Ingestion

In [ ]:
# Imports list
import math
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

from sklearn.preprocessing import OrdinalEncoder

In [ ]:
# Style is important
sns.set(style="white")

# Ensuring pandas always prints all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 1000)

%config InlineBackend.figure_format = 'retina'

In [ ]:
# Reading the csv data

data = pd.read_csv('DM2425_ABCDEats_DATASET.csv')

# 2. Wrangling the data 

## 2.1. Info()

In [ ]:
data.info()

This dataset features has 31888 observations accross 56 variables, of which, 24 refer to hours of day, and 7 to days of week. Given the descriptions provided, we would expect all of these variables to take on integer values which is not the case. Moreover, missing values are present, specifically in 'HR_0', 'customer_age' and 'first_order'.

Additionally, 'Customer_age' and 'first_order' should be integers, but are being shown in info() as floats, and the variable 'is_chain', which should be a boolean is, in fact, and integer.

In [ ]:
data.tail()

Calling .tail() shows us that no aggregations are present at the bottom of the dataset, which is good. Moreover, the data itself is just as we expected from inspecting .info().

## 2.2 Renaming variables

We rename some variables to an easier and shorter convention.

In [ ]:
_rename_dict = {
        'customer_region' : 'cust_region'
        , 'payment_method' : 'pay_method'
        , 'customer_age' : 'cust_age'
        , 'vendor_count' : 'n_vendor'
        , 'product_count' : 'n_product'
        , 'n_order' : 'n_order'
        , 'is_chain' : 'n_chain'
        , 'CUI_American' : 'American'
        , 'CUI_Asian' : 'Asian'
        , 'CUI_Beverages' : 'Beverages'
        , 'CUI_Cafe' : 'Cafe'
        , 'CUI_Chicken Dishes' : 'Chicken Dishes'
        , 'CUI_Chinese' : 'Chinese'
        , 'CUI_Desserts' : 'Desserts'
        , 'CUI_Healthy' : 'Healthy'
        , 'CUI_Indian' : 'Indian'
        , 'CUI_Italian' : 'Italian'
        , 'CUI_Japanese' : 'Japanese'
        , 'CUI_Noodle Dishes' : 'Noodle Dishes'
        , 'CUI_OTHER' : 'OTHER'
        , 'CUI_Street Food / Snacks' : 'Street Food / Snacks'
        , 'CUI_Thai' : 'Thai'
}

# Rename the columns for easier reference
data.rename(columns=_rename_dict, inplace=True)

## 2.3 Investigating 'customer_id'

We plot the values of customer id, by manipulating the hex/bin conversion.

In [ ]:
# We convert customer_id from hexadecimal to int 
id_array = data['customer_id'].apply(lambda x: int(x, 16))

# And plot its histogram
plt.hist(id_array, bins=120, edgecolor='black')
plt.show()

The conversion of the values of customer_id into hexadecimal, and the plot of their histogram shows us, what is very likely to be a uniform distribution, this tells us that these values are indeed either random, or randomly sampled, or both. We will not perform any further investigation into this hypothesis, and will use the hexadecimal form of customer_id as index.

### Checking for duplicate ID's

Before turning customer id into the index of the dataframe, we check if there are any repeated values, i.e., multiple entries for a customer.

In [ ]:
# Keep false, shows does not drop the duplicated values which allows visual inspection
data[data['customer_id'].duplicated(keep=False)].head()

Detected duplicate customer entries are removed.

In [ ]:
# Make the integer value of the customer hex values, the index. 

data['customer_id'] = id_array

# Set 'customer_id' as the index
data = data[~data['customer_id'].duplicated()].set_index('customer_id')

A small functional choice, because we retain gain the ability to slice with integer indices, but we can always convert them back to their respective hexdecimal values for future work.

## 2.4 Data Consistency

### 2.4.1 Missing Values

We now highlight missing values, and analyze the consistency of data with data types and business logic.

In [ ]:
data.isna().sum()

#### First order

At first glance it might seem odd that there are missing values for 'first_order' that do not have a corresponding last_order, but in actuality this points to a very obvious fact, that these customers had made their first order before we had begun pur data collection efforts.

Defined as:
- "Number of days from the start of the dataset when the customer first placed an order."

The soundest value to input is the minimum value possible i.e. 0.

In [ ]:
#data.loc[data['first_order'].isna(), 'first_order'] = 0

#### HR_0

"DOW_0 to DOW_6" encode -> Number of orders placed on each day of the week. (0 = Sunday, 6 = Saturday) <br>
"HR_0 to HR_23" encode -> of orders placed during each hour of the day. (0 = midnight, 23 = 11 PM)

Therefore, it is simply a matter of summing the DOW values, subtracting all of the HR values, and impute the result of the operation to the value of HR_0.

In [ ]:
sum_week = data[[f"DOW_{n}" for n in range(7)]].sum(axis=1)
sum_day = data[[f"HR_{n}" for n in range(24)]].sum(axis=1)

data.loc[data['HR_0'].isna(), 'HR_0'] = (sum_week - sum_day)

#### Age

We start by plotting the histogram for Age, making not of a heavy skew of the data towards the lower values of it's range.

In [ ]:
sns.histplot(data['cust_age'])
plt.show()

Now we look at pairwise plots of age with other features.

In [ ]:
# Specify the variable for which you want to plot against others
target_variable = 'cust_age'

# Create scatter plots
def scatter_plots(df, target_variable):
    
    # Get a list of all columns except the target variable
    other_variables = df.columns[df.columns != target_variable]

    n_cols = 2
    n_rows = math.ceil(len(df.columns) / n_cols)
    
    # Create scatter plots
    plt.figure(figsize=(12, 6 * n_rows))  # Adjust height based on number of rows
    for i, variable in enumerate(other_variables):
        plt.subplot(n_rows, n_cols, i + 1)  # Adjust the subplot layout
        sns.scatterplot(data=df, x=target_variable, y=variable)
        plt.title(f'Scatter plot of {target_variable} vs {variable}')
        plt.xlabel(target_variable)
        plt.ylabel(variable)
        
    plt.show()

# Call the function to create the plots
scatter_plots(data, target_variable)


The inspection of the plots above shows us that there seems to be some stochastic relation between some of these variables, and customer age. However, it is not something that we want to explore, or exploit just yet, so we will refrain from treating this missing values during this exploratory data analysis.

In [ ]:
correlations = {}

_df = data.loc[~data['cust_age'].isna(),].copy(deep=True)

for _x in data.columns:
    try: 
        coef, p_value  = pearsonr(_df[_x], _df['cust_age'])
        correlations[_x] = {'coef' : coef, 'p_value' : p_value}
    except Exception as e: 
        pass
        
correlations

Going one step further we can see that there is no evidence of linear association between other metric variables and customer_age, because most p-values are too large, and of those that aren't the effect is both likely spurious, as well as the actual values of the correlations insignificant. 

Taking all this into consideration, we might consider some kind of imputation later, that makes use of this knowledge that we have gathered, about these distributions. But for now we drop these values.

In [ ]:
data = data.dropna(subset='cust_age')

### 2.4.2 Unique Values

Moving on to unique values, we use this point in the notebook to assess the quality, and consistency of our variables data types, with the their expected values, given the metadata and general knowledge about the features they capture..

In [ ]:
pd.DataFrame(
    {
        'feature_type': [data[column].dtype for column in data.columns],
        'unique_values': data.apply(lambda col: sorted(pd.Series.unique(col).tolist()))
    }
)

The analysis of the output above leads us to the following conclusions:

- customer_region - value '-' possibly meaning 'unknown_region'.
- last_promo - value '-' possibly meaning 'NOPROMO'
- is_chain - has 60 different values, it may be storing the count of orders made to chain restaurants.

Other than that, all variables, now including HR_0 seem to take on values we would expect.

### 2.4.3 Inconsistencies

It can immediatly be seen that there are no negative values in any of the series above (as we have taken care in sorting them), however, many other possible types of inconsistencies are possible, yet we are confident that there are likely mostly two kinds of inconsistencies left: 

- duplicate values;
- relational impossibilities.

Duplicated values are observations that share every columnwise value; while relational impossibilities are data patterns that are inconsistent with the possible business outcomes.

#### a) Indentifying and treating duplicated values

In [ ]:
len(data[data.duplicated(keep=False)])

In [ ]:
len(data[data.duplicated()])

In [ ]:
data[data.duplicated(keep=False)].head()

We find 94 such rows, regarding 47 entries; and simply drop the repeats.

In [ ]:
data = data.drop_duplicates() 

#### b) Identifying inconsistent values

Using DeMorgan's Law and the fact that we are interested only in situations where the conditions below are all True we 
may represent the set of all inconsistent value as such:

In [ ]:
# has at least one vendor
has_vendor = data['n_vendor'] != 0 

# has at least one product
has_product = data['n_product'] != 0 

# purchase must have been made on a valid dow
some_day = (data[[f"DOW_{n}" for n in range(7)]] != 0).any(axis = 1) 

# purchase must have been made at a valid hour
some_hour = (data[[f"HR_{n}" for n in range(24)]] != 0).any(axis = 1)  

# some type of cuisine must have been ordered
some_food = (data[data.columns[9:24]] != 0).any(axis = 1) 

# We find 156 such values
data[~(has_vendor & has_product & some_day & some_hour & some_food)].shape[0] 

In [ ]:
data[~(has_vendor & has_product & some_day & some_hour & some_food)].head()

In [ ]:
data = data[(has_vendor & has_product & some_day & some_hour & some_food)] # And we drop these values

In [ ]:
# We check if the sum of the order counts for DOW, match the sum of the order counts per HR.

sum_week = data[[f"DOW_{n}" for n in range(7)]].sum(axis=1)
sum_day = data[[f"HR_{n}" for n in range(24)]].sum(axis=1)

data[(sum_day != sum_week)].shape[0] # Bam

In [ ]:
# We check if there are any customers, for whom 'last order' was placed before 'first_order'.

data[data['last_order'] < data['first_order']].shape[0] # Double Bam

We correct the inconsistent categorical values.

### 2.4.4. Treating Inconsistent values

We now analyise the nature of the values '-' in last promo and in cust_region, and 'last_promo', to see if what type of missing information they hold.

In [ ]:
# Some preparation

selected_features = data.columns.tolist()
selected_features.remove('pay_method')
selected_features.remove('last_promo')
selected_features.remove('cust_region')

#### Cust Region

Even though fancier methods exists, we apply a few simple tools learned from Statistics.  

In [ ]:
#We start by visually inspecting the output of describe applied to the groupby
# cust_region
data.groupby('cust_region').describe().T

In [ ]:
# Get unique regions
unique_regions = data['cust_region'].unique()

# Initialize a list to store results
results = []

# Iterate over all combinations of regions
for region_a, region_b in itertools.combinations(unique_regions, 2):
    # Filter data by category for each region
    group_a = data[data['cust_region'] == region_a]
    group_b = data[data['cust_region'] == region_b]
    
    # Run t-tests for each feature and store the p-values
    p_values = {}
    for feature in selected_features:  # list of continuous features
        t_stat, p_val = ttest_ind(group_a[feature], group_b[feature], equal_var=False)
        p_values[feature] = p_val
    
    # Apply Bonferroni correction
    p_values_corrected = multipletests(list(p_values.values()), method='bonferroni')
    
    # Extract adjusted p-values
    adjusted_p_values = dict(zip(p_values.keys(), p_values_corrected[1]))

    # Calculate Bonferroni threshold
    alpha = 0.05  # Original significance level
    n_tests = len(p_values)  # Number of tests
    bonferroni_threshold = alpha / n_tests

    # Create a list of features where we reject the null hypothesis
    rejected_features = [feature for feature, adjusted_p in adjusted_p_values.items() if adjusted_p < bonferroni_threshold]

    # Store results in the list as a single entry per region pair
    results.append({
        'region_a': region_a,
        'region_b': region_b,
        'rejected_features': rejected_features  # List of features rejected
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results DataFrame
results_df


While we would not present these results the juri of a murder trial - as has regretebly happened before - these multiple t-tests with bonferroni correction are useful for clustering purposes, as we are just looking for sufficient confidence in our groupings to perform exploratory analysis, moreover, if perhaps, the subsequent analysis holds we might even end up retaining them. 

Moving on, our analysis points us to the following inferences: 
- '2440' and '2490' map to the same region, as do '-' and '8670', as we had initially conjecture, as they do not differ in a statistically significant fashion in the multiple t-tests
- with only 13 observations, customer region '8550' is very difficult to group with any other, because we lack a statistically representative sample of that population;
- regions starting with the same integer differ only by a small subset of features, and as we are given the information that these pertain to three cities, we are left to conclude, that these customer regions, respect a city wide macro-trend, while they show ethno-demographic differences at a more micro-level.

We then decide to do as follows: 
- fill the values of '-' with 8670, and join region '2440' with '2490', as both cases fail to demonstrate statistical evidence of belonging to different populations;
- create a variable city, with value corresponding to the first digit of each 'cust region';
- give no focus to region '8550' in further exploratory data analysis.

In [ ]:
# We make the changes official

data.loc[data['cust_region'] == '-', 'cust_region'] = '8670'
data.loc[data['cust_region'].isin(['2440', '2490']), 'cust_region'] = '2400'

# And add the variable
data['cust_city'] = data['cust_region'].apply(lambda x: x[0])

In [ ]:
# Very good 

data['cust_city'].value_counts()

#### Last Promo

In [ ]:
# Get unique promos
unique_regions = data['last_promo'].unique()

# Initialize a list to store results
results = []

# Iterate over all combinations of regions
for region_a, region_b in itertools.combinations(unique_regions, 2):
    # Filter data by category for each region
    group_a = data[data['last_promo'] == region_a]
    group_b = data[data['last_promo'] == region_b]
    
    # Run t-tests for each feature and store the p-values
    p_values = {}
    for feature in selected_features:  # list of continuous features
        t_stat, p_val = ttest_ind(group_a[feature], group_b[feature], equal_var=False)
        p_values[feature] = p_val
    
    # Apply Bonferroni correction
    p_values_corrected = multipletests(list(p_values.values()), method='bonferroni')
    
    # Extract adjusted p-values
    adjusted_p_values = dict(zip(p_values.keys(), p_values_corrected[1]))

    # Calculate Bonferroni threshold
    alpha = 0.05  # Original significance level
    n_tests = len(p_values)  # Number of tests
    bonferroni_threshold = alpha / n_tests

    # Create a list of features where we reject the null hypothesis
    rejected_features = [feature for feature, adjusted_p in adjusted_p_values.items() if adjusted_p < bonferroni_threshold]

    # Store results in the list as a single entry per region pair
    results.append({
        'promo_a': region_a,
        'promo_b': region_b,
        'rejected_features': rejected_features  # List of features rejected
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results DataFrame
results_df


In [ ]:
data.groupby('last_promo').describe().T

Given that these correspond to 

https://merchants.ubereats.com/gb/en/resources/articles/10-types-of-sales-promotions/

## 2.5 Asserting Data types

And assert the desired datatypes for each of our variables.

In [ ]:
# Tidying up datatypes

for col in data.iloc[:, 0:9]:
    if col in ['last_promo', 'pay_method', 'cust_region']:
        data[col] = data[col].astype(object)
    else:
        data[col] = data[col].astype('Int64')

for col in data.iloc[:, 9:24]:
    data[col] = data[col].astype(float)

for col in data.iloc[:, 24:]:
    data[col] = data[col].astype('Int64')
    

In [ ]:
data.info()

In [ ]:
data.head()

This concludes the wrangling portion of our data analysis, the next portion can be found at "initial_explore.ipyjn".

In [ ]:
# Export DataFrame as a CSV file

data.reset_index(drop=False).to_csv('wrangled_data.csv', index=False)
